# Dependencies

In [1]:
# Installing opencv
!pip install opencv-python

In [1]:
# Import opencv
import cv2 

# Import uuid
import uuid

# Import Operating System
# For file paths in different os's
import os

# Import time
import time

In [1]:
# Setup Folders 

In [8]:
# Object types
labels = ['BlackCandy', 'GreenCandy', 'WhiteCandy', 'YellowCandy']
labels = ['GreenCandy']
number_imgs = 200

In [9]:
# Setitng up images path
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [10]:
# For linux (posix) copied from github
# Creating folders for each class
if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':
        !mkdir -p {IMAGES_PATH}
    if os.name == 'nt':
         !mkdir {IMAGES_PATH}
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}

# Capture Images

In [11]:
# Connect to camera, loop trough all the labels for each label take number_imgs number of images
for label in labels:
    cap = cv2.VideoCapture(1)
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(4)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# Capture Images - basler

In [ ]:
# Same as above, but for basler camera
# bayer rg8
import pypylon.p# Capture Images# Capture Imagesylon as pylon
import cv2

try:

    # We can optionally define a Device Info Object to use in finding a specific camera
    info = pylon.DeviceInfo()
    info.SetSerialNumber('23029762')

    # The InstantCamera is generic (interface agnostic)
    # It encapsulates the physical camera and driver into one convinient object
    # Open the first camera found that matches the Device Info Object (optional)
    camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice(info))

    print("Using Camera : ", camera.GetDeviceInfo().GetFriendlyName())

    # Open the camera to access features
    camera.Open()

    # We use the Genicam API to change features via strings
    # because we are programming generically
    camera.GetNodeMap().GetNode('Width').SetValue(3088)
    camera.GetNodeMap().GetNode('Height').SetValue(2064)
    #camera.PixelFormat = "BayerRG8"

    
    scale_percent = 40 # percent of original size

    # Driver/Grabber features are natively supported by the InstantCamera API
    # Because they are common to all camera interfaces
    camera.MaxNumBuffer.SetValue(20) # For demo only. Default is 10.

    # Start grabbing images.
    # The camera's physical acquisition starts, and the driver's streamgrabber is configured (buffers created, etc.)
    # StartGrabbingMax(numImages) will call StopGrabbing() automatically when numImages are retrieved.
    #while(true):
        
    time.sleep(5)
    counter = 0
    
    for label in labels:
        print('Collecting images for {}'.format(label))
        for imgnum in range(number_imgs):
            print('Collecting image {}'.format(imgnum))
            camera.StartGrabbingMax(1)

            while camera.IsGrabbing():

                # Retrieve a GrabResult from the driver
                result = camera.RetrieveResult(5000,pylon.TimeoutHandling_ThrowException)

                # The GrabResult is a container. It could hold a good image, corrupt image, no data, etc.
                if result.GrabSucceeded:
                    #print ("")
                    #print ("Grab Result Succeeded! We have an Image!")
                    #print (" Image number              : ", result.GetBlockID())
                    #print (" Dimensions                : ", result.GetWidth(), "x", result.GetHeight())
                    buffer = result.GetBuffer()
                    #print (" Gray value of first pixel : ", buffer[0])

                    # Display using OpenCV (optional for this example)
                    # imshow supports only 1,3,4 bytes/pixel. YUV is 2. So we convert to BGR
                    if pylon.IsYUV(result.GetPixelType()):
                        myImage = cv2.cvtColor(result.GetArray(), cv2.COLOR_YUV2BGR_YUYV )
                        #print ("using isyuv method")
                    else:
                        myImage = cv2.cvtColor(result.GetArray(), cv2.COLOR_BAYER_RG2RGB) # GetArray() returns a numpy array suitable for OpenCV
                        #print("not using isyuv")

                    width = int(myImage.shape[1] * scale_percent / 100)
                    height = int(myImage.shape[0] * scale_percent / 100)
                    dim = (width, height)

                    resized = cv2.resize(myImage, dim, interpolation = cv2.INTER_AREA)
                    imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
                    cv2.imwrite(imgname, resized)
                    cv2.imshow('', resized)
                    cv2.waitKey(1)

                #else:
                    #print ("")
                    #print ("Grab Result Failed!")
                    #print (" Error Description : ", result.GetErrorDescription())
                    #print (" Error Code        : ", result.GetErrorCode())

            print ("")
            print ("finished!")

            cv2.waitKey(0)
            cv2.destroyAllWindows()
    
except pylon.GenericException as err:
    print("Pylon error: {0}".format(err))

# Labelling

In [17]:
# Installing prerequisites for labelimg
!pip install --upgrade pyqt5 lxml

In [18]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

In [19]:
# Clone the labelimage github into folder
if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

In [20]:
# Instructions from labelimage
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [27]:
!cd {LABELIMG_PATH} && python labelImg.py

Image:C:\Users\Dimbo\Desktop\MxedCndy\241629321_967084880506154_5757955282905829290_n.jpg -> Annotation:C:/Users/Dimbo/Desktop/MxedCndy/241629321_967084880506154_5757955282905829290_n.xml
Image:C:\Users\Dimbo\Desktop\MxedCndy\241748666_1011337222976786_1781842510539338679_n.jpg -> Annotation:C:/Users/Dimbo/Desktop/MxedCndy/241748666_1011337222976786_1781842510539338679_n.xml
Image:C:\Users\Dimbo\Desktop\MxedCndy\241837255_410323350665947_5048049313318952450_n.jpg -> Annotation:C:/Users/Dimbo/Desktop/MxedCndy/241837255_410323350665947_5048049313318952450_n.xml
Image:C:\Users\Dimbo\Desktop\MxedCndy\241841237_2391171887687031_4877512896499750036_n.jpg -> Annotation:C:/Users/Dimbo/Desktop/MxedCndy/241841237_2391171887687031_4877512896499750036_n.xml
Image:C:\Users\Dimbo\Desktop\MxedCndy\241865798_1076507279821953_5222170422717434218_n.jpg -> Annotation:C:/Users/Dimbo/Desktop/MxedCndy/241865798_1076507279821953_5222170422717434218_n.xml
Cancel creation.
Image:C:\Users\Dimbo\Desktop\MxedCndy

qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sani

# Colab Training

In [20]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')
ARCHIVE_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'archive.tar.gz')

In [21]:
!tar -czf {ARCHIVE_PATH} {TRAIN_PATH} {TEST_PATH}